# WEB SCRAPER TO DOWNLOAD REVIEWS FROM ZOMATO

In [ ]:
#IMPORTING ALL THE REQUIRED MODULES FOR THIS TASK
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
#USER AGENT HEADERS
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
#SCRAPING DATA ONLY FROM INDIAN RESTAURENTS
url="http://www.zomato.com/india"
#STORES LINKS
links=[]
eats=["lunch","breakfast","dinner","delivery"]
#First write links to it.
f=open("hotellinks.txt","w")
#open the above file in read mode after getting all the hotel links
#f=open("hotellinks.txt","r")
#hotels=f.readlines()
hotels=[]
#uncomment these while scraping reviews
#for hotel in f.readlines():
#    hotels.append(hotel.strip("\n"))  
def get_links():
    temp=[]
    r=requests.get(url,headers=headers)
    bs=BeautifulSoup(r.content,"lxml")
    for href in bs.find_all("div",class_="ui segment"):
        temp.append(href.find_all('a')[0]['href'].strip(""))
    for i in range(5):
        for link in temp:
            if re.search(r'/$',link):
                temp.remove(link)
    for link in temp:
        for eat in eats:
            links.append(link+"/"+eat)
def get_hotels():
    for link in links:
        time.sleep(2)
        try:
            r=requests.get(link,headers=headers)
        except:
            pass
        bs=BeautifulSoup(r.content,"lxml")
        for link in bs.find_all("a",{"data-result-type":"ResCard_Name"}):
            f.write(link['href'])
            f.write("\n")
            hotels.append(link['href'])

def get_reviews():
    reviews_dict={"Reviews":[],"Rating":[],"label":[],"hotel":[]}
    sleeptime=0
    for hotel in hotels:
        time.sleep(1)
        sleeptime=sleeptime+1
        try:
            r=requests.get(hotel,headers=headers)
        except Exception as e:
            print(e)
            pass
        bs=BeautifulSoup(r.content,"lxml")
        for text in bs.find_all("div",{"itemprop":"description"}):
            print(hotel)
            if(hotel.count("/")==3):
                reviews_dict["hotel"].append(hotel.split("/")[3])
            else:
                 reviews_dict["hotel"].append(hotel.split("/")[4])   
            reviews_dict["Reviews"].append(text.text.splitlines()[2].strip(" "))
            reviews_dict["Rating"].append(text.find("div")['aria-label'].strip("Rated"))
            reviews_dict["label"].append(text.find("div")['title'])
            #print(text.text,text.find("div")['title'],text.find("div")['aria-label'].strip("Rated").strip(" "))
        if(sleeptime%1000==0):
            time.sleep(5)
        data= pd.DataFrame.from_dict(reviews_dict)
        data.to_csv("data2.csv",headers=False)
#Uncomment these lines to work your way through scraping    
#get_links()
#get_hotels()
#get_reviews()
#print(hotels[2426])
f.close()



# Script for getting more reviews through XHR requests

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd
def more_reviews():
    s=requests.session()
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    for i,hotel in enumerate(hotels):
        print(hotel)
        try:
            r=s.get(hotel,headers=headers)
        except:
            pass
        b=BeautifulSoup(r.content,"lxml")
        if b.find("span",class_="zs-load-more-count"):
            total=int(b.find("span",class_="zs-load-more-count").text)
            #print(r.content)
            #print(b.find_all("div",class_="ui segment clearfix zs-load-more res-page-load-more")[0]["data-entity_id"])
            my_cookie={"__jpuri":hotel}
            headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                     "Referer":"https://www.zomato.com/"
                    }
            requests.utils.add_dict_to_cookiejar(s.cookies, my_cookie)
            for number in range(2,total):
                payload={"entity_id":b.find_all("div",class_="ui segment clearfix zs-load-more res-page-load-more")[0]["data-entity_id"],
                          "limit":"5",
                          "page":number,
                          "profile_action":"reviews-top"}
                try:
                    req=s.post("https://www.zomato.com/php/social_load_more.php",headers=headers,data=payload)
                except:
                    pass
                j=json.loads(req.content.decode("utf-8").strip("\n"))
                html=j['html'].strip("\n")
                bs=BeautifulSoup(html,"lxml")
                #print(bs.find_all("div",{"itemprop":"description"}))
                for text in bs.find_all("div",{"itemprop":"description"}):
                    if(hotel.count("/")==3):
                        reviews_dict["hotel"].append(hotel.split("/")[3])
                    else:
                        reviews_dict["hotel"].append(hotel.split("/")[4])  
                    reviews_dict["Reviews"].append(text.text.splitlines()[2].strip(" "))
                    reviews_dict["Rating"].append(text.find("div")['aria-label'].strip("Rated"))
                    reviews_dict["label"].append(text.find("div")['title'])
                    #print(text.text,text.find("div")['title'],text.find("div")['aria-label'].strip("Rated").strip(" "))
            data= pd.DataFrame.from_dict(reviews_dict)
            #print("something")
            data.to_csv("data.csv",headers=False)
f=open("hotellinks.txt","r")
hotels=[]
reviews_dict={"Reviews":[],"Rating":[],"label":[],"hotel":[]}
for h in f.readlines():
    hotels.append(h.strip("\n"))
#uncomment below line 
#more_reviews()
